In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
import graphviz as gv
from sklearn.tree import export_graphviz
from sklearn.model_selection import KFold
from sklearn.cluster import KMeans
import datetime
import sklearn.metrics as skmetrics
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV

In [2]:
data = pd.read_csv('./data/loan_prediction_train.csv')

In [3]:
data['Dependents'] = data['Dependents'].fillna(0)

In [4]:
#data[data['Dependents'] == '3+']

In [5]:
data['Dependents'].replace('3+','3').astype(float)

0      0.0
1      1.0
2      0.0
3      0.0
4      0.0
      ... 
609    0.0
610    3.0
611    1.0
612    2.0
613    0.0
Name: Dependents, Length: 614, dtype: float64

In [6]:
#data[data['Self_Employed'] == '3+']

In [7]:
data[data['Self_Employed'] == '']

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status


In [8]:
data['Self_Employed'].replace('','No')

0       No
1       No
2      Yes
3       No
4       No
      ... 
609     No
610     No
611     No
612     No
613    Yes
Name: Self_Employed, Length: 614, dtype: object

In [9]:
data[data['Self_Employed'] == '']

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status


In [10]:
data['Credit_History'] = data['Credit_History'].fillna(0)

In [11]:
data['Loan_Amount_Term'] = data['Loan_Amount_Term'].fillna(0)

In [12]:
#Encode the data
#remove loan ID
data = data[['Gender', 'Married', 'Dependents', 'Education',
       'Self_Employed', 'ApplicantIncome', 'CoapplicantIncome', 'LoanAmount',
       'Loan_Amount_Term', 'Credit_History', 'Property_Area', 'Loan_Status']]

dataEncoded = pd.get_dummies(data,columns = ['Gender'], drop_first=True)

dataEncoded = pd.get_dummies(dataEncoded,columns = ['Married'], drop_first=True)

dataEncoded = pd.get_dummies(dataEncoded,columns = ['Education'], drop_first=True)

dataEncoded = pd.get_dummies(dataEncoded,columns = ['Self_Employed'], drop_first=True)

#STILL TO DO
#replace Loan_Amount_term NAN values with median 
dataEncoded = pd.get_dummies(dataEncoded,columns = ['Loan_Amount_Term'], drop_first=True)

dataEncoded = pd.get_dummies(dataEncoded,columns = ['Property_Area'], drop_first=True)

dataEncoded = pd.get_dummies(dataEncoded,columns = ['Loan_Status'], drop_first=True)





In [14]:
dataEncoded.columns

Index(['Dependents', 'ApplicantIncome', 'CoapplicantIncome', 'LoanAmount',
       'Credit_History', 'Gender_Male', 'Married_Yes',
       'Education_Not Graduate', 'Self_Employed_Yes', 'Loan_Amount_Term_12.0',
       'Loan_Amount_Term_36.0', 'Loan_Amount_Term_60.0',
       'Loan_Amount_Term_84.0', 'Loan_Amount_Term_120.0',
       'Loan_Amount_Term_180.0', 'Loan_Amount_Term_240.0',
       'Loan_Amount_Term_300.0', 'Loan_Amount_Term_360.0',
       'Loan_Amount_Term_480.0', 'Property_Area_Semiurban',
       'Property_Area_Urban', 'Loan_Status_Y'],
      dtype='object')

$Exploratory Testing!!!!!!!$